# Colorizing Grayscale Images Using Neural Networks

## Import the necessary libraries

In [ ]:
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import TensorBoard
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
from skimage.transform import resize
import numpy as np
import random
from PIL import Image
import requests

## Load in our training data

We will specifically focus on colorizing images of 
human faces.  First, let's focus on training our model on only one image.

In [ ]:
#url = 'https://assets.website-files.com/5d315dd990eed30aa7ca5f8f/5d3d843acb7dfd63bcf37e29_tree-and-woodland-company-landscape.jpg'
url = 'https://1vw4gb3u6ymm1ev2sp2nlcxf-wpengine.netdna-ssl.com/wp-content/uploads/shutterstock_149962697-946x658.jpg'

In [ ]:
image = Image.open(requests.get(url, stream=True).raw)
image.show()

## Preprocessing the input image

In [ ]:
image = img_to_array(image)
image = np.array(image, dtype=float)
image.shape

(658, 946, 3)

In [ ]:
input = rgb2lab(1.0/255*image)[:,:,0]
input = resize(input, (400, 400))
input = input.reshape((1, 400, 400, 1))

In [ ]:
output = rgb2lab(1.0/255*image)[:,:,1:]
output /= 128
output = resize(output, (400, 400, 2))
output = output.reshape((1, 400, 400, 2))

## Build the Neural Network

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(2, (3, 3), activation='tanh', padding='same')
])

## Compile the model

In [ ]:
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

## Train the model

In [ ]:
model.fit(x=input, y=output, batch_size=1, epochs=3000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 35ms/step - loss: 0.0033 - accuracy: 0.8261
Epoch 502/3000
1/1 [==============================] - 0s 44ms/step - loss: 0.0033 - accuracy: 0.8009
Epoch 503/3000
1/1 [==============================] - 0s 39ms/step - loss: 0.0122 - accuracy: 0.3321
Epoch 504/3000
1/1 [==============================] - 0s 39ms/step - loss: 0.0056 - accuracy: 0.8179
Epoch 505/3000
1/1 [==============================] - 0s 40ms/step - loss: 0.0047 - accuracy: 0.8141
Epoch 506/3000
1/1 [==============================] - 0s 28ms/step - loss: 0.0040 - accuracy: 0.7966
Epoch 507/3000
1/1 [==============================] - 0s 43ms/step - loss: 0.0036 - accuracy: 0.8131
Epoch 508/3000
1/1 [==============================] - 0s 33ms/step - loss: 0.0078 - accuracy: 0.3674
Epoch 509/3000
1/1 [==============================] - 0s 36ms/step - loss: 0.0052 - accuracy: 0.8236
Epoch 510/3000
1/1 [==============================] - 0

## Evaluate the model

In [ ]:
model.evaluate(input, output, batch_size=1, verbose=2)

1/1 - 0s - loss: 6.6363e-04 - accuracy: 0.6140


[0.0006636324105784297, 0.6140062212944031]

## Colorizing images using the model

Let's colorize the image we used to train the model on.

In [ ]:
result = model.predict(input)
result *= 128
colorized_img = np.zeros((400, 400, 3))
colorized_img[:,:,0] = input[0][:,:,0]
colorized_img[:,:,1:] = result[0]
imsave("original_img.png", resize(image, (400, 400, 3)))
imsave("colorized_img.png", lab2rgb(colorized_img))
imsave("grayscale_img.png", rgb2gray(lab2rgb(colorized_img)))

Even though our model was able to colorize the image pretty well, it is still a weak neural network. To understand this, try colorizing images the model has not been trained on.

In [ ]:
#url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_kjXxkqXog81CD2olSDquVyi92iu-GVCH0w&usqp=CAU'
url = 'https://546458-1766807-raikfcquaxqncofqfm.stackpathdns.com/pub/media/wordpress/d5f1425700d7460bb2aa1e1e8e1b7e49.jpg'

In [ ]:
image = Image.open(requests.get(url, stream=True).raw)
image = img_to_array(image)
image = np.array(image, dtype=float)

In [ ]:
input = rgb2lab(1.0/255*image)[:,:,0]
input = resize(input, (400, 400))
input = input.reshape(1, 400, 400, 1)

In [ ]:
result = model.predict(input)
result *= 128
colorized_img = np.zeros((400, 400, 3))
colorized_img[:,:,0] = input[0][:,:,0]
colorized_img[:,:,1:] = result[0]
imsave("original_img.png", resize(image, (400, 400, 3)))
imsave("colorized_img.png", lab2rgb(colorized_img))
imsave("grayscale_img.png", rgb2gray(lab2rgb(colorized_img)))


We will use a Kaggle dataset of [human faces]('https://www.kaggle.com/greatgamedota/ffhq-face-data-set') for our training images. If you do already have one, make a Kaggle account. Go to API --> Create New API Token. Your kaggle.json will be downloaded. Upload the file to the Colab files. 






In [ ]:
!pip install kaggle

In [ ]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!kaggle datasets download -d arnaud58/landscape-pictures
#!kaggle datasets download -d ashwingupta3012/human-faces
!kaggle datasets download -d greatgamedota/ffhq-face-data-set

 99% 1.96G/1.97G [00:20<00:00, 120MB/s]
100% 1.97G/1.97G [00:20<00:00, 102MB/s]


In [ ]:
from zipfile import ZipFile
file_name = 'ffhq-face-data-set.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall('dataset')
  print('Done')

Done


In [ ]:
img_dataset = []
for filename in os.listdir('./dataset/thumbnails128x128/')[0:100]:
    img = img_to_array(load_img('./dataset/thumbnails128x128/'+filename))
    img = resize(img, (256, 256, 3))
    img_dataset.append(img)
img_dataset = np.array(img_dataset, dtype=float)
img_dataset.shape

(100, 256, 256, 3)

In [ ]:
split = int(0.95*len(img_dataset))
train = img_dataset[:split]
train = 1.0/255*train

In [ ]:
# Image transformer
datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, rotation_range=20, horizontal_flip=True)

In [ ]:
# # Generate training data
# for batch in datagen.flow(train_input, batch_size=10):
#   lab_batch = rgb2lab(batch)
#   input_batch = lab_batch[:,:,:,0]
#   output_batch = lab_batch[:,:,:,1:] / 128
#         yield (input_batch.reshape(input_batch.shape+(1,)), output_batch)
#train = datagen.flow(train, batch_size=1)
print(type(train))
train_input = rgb2lab(1.0/255*train)[:,:,:,0]
train_input = train_input.reshape(train_input.shape + (1,))
train_output = rgb2lab(1.0/255*train)[:,:,:,1:] / 128

<class 'numpy.ndarray'>


In [ ]:
train_output.shape

(95, 256, 256, 2)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(2, (3, 3), activation='tanh', padding='same')
])

In [ ]:
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

In [ ]:
# Train model      
#model.fit(train_input, train_output, epochs=100)
model.fit(datagen.flow(train_input, train_output, batch_size=10), epochs=100, steps_per_epoch=10)


Epoch 1/100
10/10 [==============================] - 2s 84ms/step - loss: 6.0121e-06 - accuracy: 0.5549
Epoch 2/100
10/10 [==============================] - 1s 83ms/step - loss: 1.0531e-07 - accuracy: 0.5247
Epoch 3/100
10/10 [==============================] - 1s 82ms/step - loss: 1.1343e-07 - accuracy: 0.5211
Epoch 4/100
10/10 [==============================] - 1s 77ms/step - loss: 2.4409e-07 - accuracy: 0.5131
Epoch 5/100
10/10 [==============================] - 1s 77ms/step - loss: 3.8892e-07 - accuracy: 0.5210
Epoch 6/100
10/10 [==============================] - 1s 73ms/step - loss: 2.9250e-07 - accuracy: 0.4880
Epoch 7/100
10/10 [==============================] - 1s 75ms/step - loss: 2.9004e-07 - accuracy: 0.5517
Epoch 8/100
10/10 [==============================] - 1s 76ms/step - loss: 4.0294e-07 - accuracy: 0.5277
Epoch 9/100
10/10 [==============================] - 1s 77ms/step - loss: 1.9512e-07 - accuracy: 0.4944
Epoch 10/100
10/10 [==============================] - 1s 77ms/st

In [ ]:
# Test images
input_test = rgb2lab(1.0/255*img_dataset[split:])[:,:,:,0]
input_test = input_test.reshape(input_test.shape+(1,))
output_test = rgb2lab(1.0/255*img_dataset[split:])[:,:,:,1:]
output_test = output_test / 128
print(model.evaluate(input_test, output_test, batch_size=10))

1/1 [==============================] - 0s 245ms/step - loss: 0.0340 - accuracy: 0.4916
[0.034049876034259796, 0.49155884981155396]


In [ ]:
color_me = []
for filename in os.listdir('./dataset/thumbnails128x128/')[1000:1010]:
    img = img_to_array(load_img('./dataset/thumbnails128x128/'+filename))
    img = resize(img, (256, 256, 3))
    imsave("original/img_"+str(len(color_me))+".jpg", img)
    color_me.append(img)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
results = model.predict(color_me)
results = results * 128
# Output colorizations
for i in range(len(results)):
    colorized = np.zeros((256, 256, 3))
    colorized[:,:,0] = color_me[i][:,:,0]
    colorized[:,:,1:] = results[i]
    imsave("result/img_"+str(i)+".jpg", lab2rgb(colorized))

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 9 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
